Hello

This notebook is inspired from "Lightgbm(Regressor) | Kaggle" kernel.
https://www.kaggle.com/hiro5299834/wifi-features-with-lightgbm-kfold


I am studying Machine Learning and I was requested to participate to one competition of my choise on Kaggle.
So I am doing it as an exercice.
Please be indulgent.
My plan here is to capture all the big steps in the process of machine learning modeling and to test some specific models that are not yet used by other kernels on this competition.

My first thought is to test Xgboost since the primary Kernel I am inspiring on is using Lightgbm model.

I will test another model later on.

For the exploratory step I have been inspired by these notebooks:
- https://www.kaggle.com/chandrylpaternetony/data-descript-outlier-detect-floor-mapping

- https://www.kaggle.com/ihelon/indoor-location-exploratory-data-analysis/notebook


Quick overview of the input data structure

In [11]:
import os
print(f"Number of train sites {len(os.listdir('../input/indoor-location-navigation/train/'))}")
print(f"Number of train meta {len(os.listdir('../input/indoor-location-navigation/metadata/'))}")
print(f"Number of test sites {len(os.listdir('../input/indoor-location-navigation/test/'))}")

Number of train sites 204
Number of train meta 204
Number of test sites 626


In [12]:
import os

input_data_structure = list()
for site in os.listdir('../input/indoor-location-navigation/train/'):
    for floor in os.listdir('../input/indoor-location-navigation/train/' + site + '/'):
        for phone in os.listdir('../input/indoor-location-navigation/train/' + site + '/' + floor + '/'):
            input_data_structure.append([site,floor,phone])
print(len(input_data_structure))            
                                        

26925


Conversion into pandas DataFrame for quick statistics over the input data files

In [13]:
import pandas as pd

input_df = pd.DataFrame(input_data_structure, columns=['sites','floor','phone'])
input_df.describe(include="all")

,sites,floor,phone
count,26925,26925,26925
unique,204,43,26925
top,5d27075f03f801723c2e360f,F1,5d60ad2404ffc90008edf43d.txt
freq,1141,4557,1


* There are **204** different **sites**
* There are **43!!!** different **floors** identification, be aware F1 or 1F is the same floor. So we obviously have less then 43 different floors
* there are **26925** different **phones** and every phone has only one single recording and has been recording at the same floor


In [14]:
table = pd.pivot_table(input_df, values=['phone'], index=['sites','floor'], margins=True,
                    aggfunc={"phone": "count"})
print(table)

                                phone
sites                    floor       
5a0546857ecc773753327266 B1       109
                         F1       131
                         F2       110
                         F3        78
                         F4        86
...                               ...
5dc8cea7659e181adb076a3f F4        79
                         F5       103
                         F6        81
                         F7        40
All                             26925

[982 rows x 1 columns]


In [15]:
table.describe(include = "all")

,phone
count,982.000000
mean,54.837067
std,859.103814
min,1.000000
25%,6.000000
50%,14.000000
75%,31.000000
max,26925.000000


In [ ]:
table_floor = pd.pivot_table(input_df, values=['floor'], index=['sites'], margins=True,
                    aggfunc={"floor": "count"})
print(table_floor)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(table_floor.index[:-1],table_floor['floor'][:-1])

We will focus on Wifi features.
We will take for granted the preprocessing of input data **indoor-location-navigation** and its transformation into **indoor-navigation-and-location-wifi-features**

About this Dataset
Content
Version2 update:
Contains features for the indoor location and navigation competition. They were based on "Indoor Navigation and Location Wifi Features" data by @devinanzelmo and generated using only wifi's bssid in the training dataset.

In [3]:
import os

local_path = 'C:/Users/hp/Documents/OpenClassrooms/P8/'
# print(os.path)
# print(f"Number of csv inputs {len(os.listdir('../input/indoor-navigation-and-location-wifi-features/'))}")
print(f"Number of csv inputs {len(os.listdir(local_path + 'input/indoor-navigation-and-location-wifi-features/'))}")

<module 'ntpath' from 'C:\\ProgramData\\Anaconda3\\lib\\ntpath.py'>
Number of csv inputs 48


import xgboost as xgb

xgb_params = {'max_depth':2, 'eta':1, 'objective':'reg:squarederror', 'eval_metric':'rmse', 'seed': SEED }

# using scikit learn API

''' lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.1,
              'num_leaves': 90,
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }
'''
xgb_params = {'objective': 'root_mean_squared_error',
              'booster': 'gbtree',
              'n_estimators': 50000,
              'learning_rate': 0.1,              
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

# classifier
'''
xgb_f_params = {'objective': 'multiclass',
                'booster': 'gbtree',
                'n_estimators': 50000,
                'learning_rate': 0.1,                
                'reg_alpha': 10,
                'reg_lambda': 2,
                'random_state': SEED,
                'n_jobs': -1
                }
                '''

Parameters
n_estimators (int) – Number of boosting rounds.

use_label_encoder (bool) – (Deprecated) Use the label encoder from scikit-learn to encode the labels. For new code, we recommend that you set this parameter to False.

max_depth (int) – Maximum tree depth for base learners.

learning_rate (float) – Boosting learning rate (xgb’s “eta”)

verbosity (int) – The degree of verbosity. Valid values are 0 (silent) - 3 (debug).

objective (string or callable) – Specify the learning task and the corresponding learning objective or a custom objective function to be used (see note below).

booster (string) – Specify which booster to use: gbtree, gblinear or dart.

tree_method (string) – Specify which tree method to use. Default to auto. If this parameter is set to default, XGBoost will choose the most conservative option available. It’s recommended to study this option from parameters document.

n_jobs (int) – Number of parallel threads used to run xgboost. When used with other Scikit-Learn algorithms like grid search, you may choose which algorithm to parallelize and balance the threads. Creating thread contention will significantly slow down both algorithms.

gamma (float) – Minimum loss reduction required to make a further partition on a leaf node of the tree.

min_child_weight (float) – Minimum sum of instance weight(hessian) needed in a child.

max_delta_step (float) – Maximum delta step we allow each tree’s weight estimation to be.

subsample (float) – Subsample ratio of the training instance.

colsample_bytree (float) – Subsample ratio of columns when constructing each tree.

colsample_bylevel (float) – Subsample ratio of columns for each level.

colsample_bynode (float) – Subsample ratio of columns for each split.

reg_alpha (float (xgb's alpha)) – L1 regularization term on weights

reg_lambda (float (xgb's lambda)) – L2 regularization term on weights

scale_pos_weight (float) – Balancing of positive and negative weights.

base_score – The initial prediction score of all instances, global bias.

random_state (int) –

Random number seed.

In [ ]:
import pandas as pd
subm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv', index_col=0)
subm.describe(include='all')

In [2]:
subm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10133 entries, 5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009 to 5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000100447
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   floor   10133 non-null  int64  
 1   x       10133 non-null  float64
 2   y       10133 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 316.7+ KB


In [4]:
../input/indoor-navigation-and-location-wifi-features/5a0546857ecc773753327266_train.csv

4

In [7]:
import pandas as pd
# df = pd.read_csv('../input/indoor-navigation-and-location-wifi-features/5a0546857ecc773753327266_test.csv',index_col=0)
# df = pd.read_csv('../input/indoor-navigation-and-location-wifi-features/5a0546857ecc773753327266_train.csv',index_col=0)
df = pd.read_csv('../input/indoor-navigation-and-location-wifi-features/5da138314db8ce0c98bbf3a0_train.csv',index_col=0)
# df = pd.read_csv('../input/indoor-navigation-and-location-wifi-features/5d2709a003f801723c3251bf_train.csv',index_col=0)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9012 entries, 4 to 4
Columns: 1216 entries, 00505a2c465e4a7f52e64beb1902c9aee8b04a90 to path
dtypes: float64(2), int64(1213), object(1)
memory usage: 83.7+ MB


In [6]:
# ------------------------------------------------------------------------------
# Import libraries
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob

from sklearn.model_selection import KFold
# import lightgbm as lgb
import xgboost as xgb

import psutil
import random
import os
import time
import sys
import math
from contextlib import contextmanager


# 
local_path = 'C:/Users/hp/Documents/OpenClassrooms/P8/'


# ------------------------------------------------------------------------------
# Fixed values
# ------------------------------------------------------------------------------
N_SPLITS = 10
SEED = 42

# ------------------------------------------------------------------------------
# File path definition
# ------------------------------------------------------------------------------
# LOG_PATH = Path("./log/")
LOG_PATH = Path(local_path + "log/")
LOG_PATH.mkdir(parents=True, exist_ok=True)


# ------------------------------------------------------------------------------
# Utilities
# ------------------------------------------------------------------------------
@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]


def score_log(df: pd.DataFrame, num_files: int, nam_file: str, data_shape: tuple, n_fold: int, seed: int, mpe: float):
    score_dict = {'n_files': num_files, 'file_name': nam_file, 'shape': data_shape, 'fold': n_fold, 'seed': seed, 'score': mpe}
    # noinspection PyTypeChecker
    df = pd.concat([df, pd.DataFrame.from_dict([score_dict])])
    df.to_csv(LOG_PATH / f"log_score.csv", index=False)
    return df




# ------------------------------------------------------------------------------
# Set seed
# ------------------------------------------------------------------------------
set_seed(SEED)

# ------------------------------------------------------------------------------
# Read data
# ------------------------------------------------------------------------------
# feature_dir = "../input/indoor-navigation-and-location-wifi-features"
feature_dir = local_path  + "input/indoor-navigation-and-location-wifi-features"

train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
print("There are {} *_train.csv files".format(len(train_files)))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))
print("There are {} *_test.csv files".format(len(test_files)))

subm = pd.read_csv(local_path + 'input/submission/sample_submission.csv', index_col=0)
# subm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv', index_col=0)

# ------------------------------------------------------------------------------
# Define parameters for models
# ------------------------------------------------------------------------------
lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.1,
              'num_leaves': 90,
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }
# add settings for xgboost
xgb_params = {'objective': 'reg:squarederror',
              'booster': 'gbtree',
              'n_estimators': 1000,
              'learning_rate': 0.1,              
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

lgb_f_params = {'objective': 'multiclass',
                'boosting_type': 'gbdt',
                'n_estimators': 50000,
                'learning_rate': 0.1,
                'num_leaves': 90,
                'colsample_bytree': 0.4,
                'subsample': 0.6,
                'subsample_freq': 2,
                'bagging_seed': SEED,
                'reg_alpha': 10,
                'reg_lambda': 2,
                'random_state': SEED,
                'n_jobs': -1
                }
# WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
# UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

xgb_f_params = {'objective': 'multi:softmax', #multi:softmax  multi:softprob, will require use of encoder of type integer
                'booster': 'gbtree',               
                'n_estimators': 1000,
                'learning_rate': 0.1,                
                'reg_alpha': 10,
                'reg_lambda': 2,
                'random_state': SEED,                
                'n_jobs': -1
                }
# ------------------------------------------------------------------------------
# Training and inference
# ------------------------------------------------------------------------------
score_df = pd.DataFrame()
oof = list()
predictions = list()
for n_files, file in enumerate(train_files):
    # trick here is to use single "for loop" to process both train and test datasets
    print("process training set for site file num {}".format(n_files))
    data = pd.read_csv(file, index_col=0)
    print("process testing set for site file num {}".format(n_files))
    test_data = pd.read_csv(test_files[n_files], index_col=0)

    oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
    preds_x, preds_y = 0, 0
    preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-4])):
        X_train = data.iloc[trn_idx, :-4]
        y_trainx = data.iloc[trn_idx, -4]
        y_trainy = data.iloc[trn_idx, -3]
        y_trainf = data.iloc[trn_idx, -2]

        X_valid = data.iloc[val_idx, :-4]
        y_validx = data.iloc[val_idx, -4]
        y_validy = data.iloc[val_idx, -3]
        y_validf = data.iloc[val_idx, -2]

        # modelx = lgb.LGBMRegressor(**lgb_params)
        modelx = xgb.XGBRegressor(**xgb_params)
        with timer("fit X"):
            modelx.fit(X_train, y_trainx,
                       eval_set=[(X_valid, y_validx)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                      )

        # modely = lgb.LGBMRegressor(**lgb_params)
        modely = xgb.XGBRegressor(**xgb_params)
        with timer("fit Y"):
            modely.fit(X_train, y_trainy,
                       eval_set=[(X_valid, y_validy)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       )

        # modelf = lgb.LGBMClassifier(**lgb_f_params)
        # remove param eval_metric='multi_logloss', for xgb
        modelf = xgb.XGBClassifier(**xgb_f_params)
        with timer("fit F"):
            modelf.fit(X_train, y_trainf,
                       eval_set=[(X_valid, y_validf)],
                       eval_metric='mlogloss', #for xgboot
                       verbose=False,
                       early_stopping_rounds=20
                       )

        oof_x[val_idx] = modelx.predict(X_valid)
        oof_y[val_idx] = modely.predict(X_valid)
        oof_f[val_idx] = modelf.predict(X_valid).astype(int)
        
        with timer("Pred X"):
            preds_x += modelx.predict(test_data.iloc[:, :-1]) / N_SPLITS
            
        with timer("Pred Y"):
            preds_y += modely.predict(test_data.iloc[:, :-1]) / N_SPLITS
        
        with timer("Pred F"):
            preds_f_arr[:, fold] = modelf.predict(test_data.iloc[:, :-1]).astype(int)

        score = comp_metric(oof_x[val_idx], oof_y[val_idx], oof_f[val_idx],
                            y_validx.to_numpy(), y_validy.to_numpy(), y_validf.to_numpy())
        print(f"fold {fold}: mean position error {score}")
        score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, fold, SEED, score)

    print("*+"*40)
    print(f"file #{n_files}, shape={data.shape}, name={os.path.basename(file)}")
    score = comp_metric(oof_x, oof_y, oof_f,
                        data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy(), data.iloc[:, -2].to_numpy())
    oof.append(score)
    print(f"mean position error {score}")
    print("*+"*40)
    score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, 999, SEED, score)

    preds_f_mode = stats.mode(preds_f_arr, axis=1)
    preds_f = preds_f_mode[0].astype(int).reshape(-1)
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    test_preds.columns = subm.columns
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    predictions.append(test_preds)

# ------------------------------------------------------------------------------
# Submit the result
# ------------------------------------------------------------------------------
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)
all_preds.to_csv('submission.csv')

There are 24 *_train.csv files
There are 24 *_test.csv files
process training set for site file num 0
process testing set for site file num 0


[0.1GB(-0.4GB): 607.898sec] fit X
[0.3GB(+0.2GB): 606.982sec] fit Y
[0.3GB(+0.0GB): 271.742sec] fit F
[0.3GB(+0.0GB): 0.584sec] Pred X
[0.3GB(+0.0GB): 0.220sec] Pred Y
[0.3GB(+0.0GB): 0.240sec] Pred F


fold 0: mean position error 3.72969839558177


[0.5GB(+0.0GB): 602.802sec] fit X
[0.5GB(+0.0GB): 601.672sec] fit Y
[0.4GB(-0.1GB): 284.680sec] fit F
[0.4GB(+0.0GB): 0.251sec] Pred X
[0.4GB(+0.0GB): 0.254sec] Pred Y
[0.4GB(+0.0GB): 0.232sec] Pred F


fold 1: mean position error 3.7192771108315843


...

[0.5GB(+0.0GB): 561.327sec] fit X
[0.4GB(-0.1GB): 616.288sec] fit Y
[0.3GB(-0.1GB): 324.655sec] fit F
[0.3GB(+0.0GB): 1.933sec] Pred X
[0.3GB(+0.0GB): 0.128sec] Pred Y
[0.3GB(+0.0GB): 0.141sec] Pred F


fold 8: mean position error 4.77142754632669


[0.5GB(+0.0GB): 582.142sec] fit X
[0.4GB(-0.1GB): 563.902sec] fit Y
[0.4GB(-0.0GB): 288.926sec] fit F
[0.4GB(+0.0GB): 0.382sec] Pred X
[0.4GB(+0.0GB): 0.119sec] Pred Y
[0.4GB(+0.0GB): 0.124sec] Pred F


fold 9: mean position error 4.6598225591795535
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
file #1, shape=(9737, 3067), name=5c3c44b80379370013e0fd2b_train.csv
mean position error 4.717837075733007
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
process training set for site file num 2
process testing set for site file num 2


MemoryError: Unable to allocate 1.12 GiB for an array with shape (7029, 21299) and data type int64

In [9]:
# for SVM and SVR

# ------------------------------------------------------------------------------
# Import libraries
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob

from sklearn.model_selection import KFold
# import lightgbm as lgb
import xgboost as xgb

# https://scikit-learn.org/stable/auto_examples/svm/plot_svm_regression.html#sphx-glr-auto-examples-svm-plot-svm-regression-py
from sklearn.svm import SVR, SVC

import psutil
import random
import os
import time
import sys
import math
from contextlib import contextmanager


# 
local_path = 'C:/Users/hp/Documents/OpenClassrooms/P8/'


# ------------------------------------------------------------------------------
# Fixed values
# ------------------------------------------------------------------------------
N_SPLITS = 10
SEED = 42

# ------------------------------------------------------------------------------
# File path definition
# ------------------------------------------------------------------------------
# LOG_PATH = Path("./log/")
LOG_PATH = Path(local_path + "log/")
LOG_PATH.mkdir(parents=True, exist_ok=True)


# ------------------------------------------------------------------------------
# Utilities
# ------------------------------------------------------------------------------
@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]


def score_log(df: pd.DataFrame, num_files: int, nam_file: str, data_shape: tuple, n_fold: int, seed: int, mpe: float):
    score_dict = {'n_files': num_files, 'file_name': nam_file, 'shape': data_shape, 'fold': n_fold, 'seed': seed, 'score': mpe}
    # noinspection PyTypeChecker
    df = pd.concat([df, pd.DataFrame.from_dict([score_dict])])
    df.to_csv(LOG_PATH / f"log_score.csv", index=False)
    return df




# ------------------------------------------------------------------------------
# Set seed
# ------------------------------------------------------------------------------
set_seed(SEED)

# ------------------------------------------------------------------------------
# Read data
# ------------------------------------------------------------------------------
# feature_dir = "../input/indoor-navigation-and-location-wifi-features"
feature_dir = local_path  + "input/indoor-navigation-and-location-wifi-features"

train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
print("There are {} *_train.csv files".format(len(train_files)))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))
print("There are {} *_test.csv files".format(len(test_files)))

subm = pd.read_csv(local_path + 'input/submission/sample_submission.csv', index_col=0)
# subm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv', index_col=0)

# ------------------------------------------------------------------------------
# Define parameters for models
# ------------------------------------------------------------------------------

# add settings for xgboost
xgb_params = {'objective': 'reg:squarederror',
              'booster': 'gbtree',
              'n_estimators': 1000,
              'learning_rate': 0.1,              
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }


# WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
# UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

xgb_f_params = {'objective': 'multi:softmax', #multi:softmax  multi:softprob, will require use of encoder of type integer
                'booster': 'gbtree',               
                'n_estimators': 1000,
                'learning_rate': 0.1,                
                'reg_alpha': 10,
                'reg_lambda': 2,
                'random_state': SEED,                
                'n_jobs': -1
                }
# ------------------------------------------------------------------------------
# Training and inference
# ------------------------------------------------------------------------------
score_df = pd.DataFrame()
oof = list()
predictions = list()
for n_files, file in enumerate(train_files):
    # trick here is to use single "for loop" to process both train and test datasets
    print("process training set for site file num {}".format(n_files))
    data = pd.read_csv(file, index_col=0)
    print("process testing set for site file num {}".format(n_files))
    test_data = pd.read_csv(test_files[n_files], index_col=0)

    oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
    preds_x, preds_y = 0, 0
    preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-4])):
        X_train = data.iloc[trn_idx, :-4]
        y_trainx = data.iloc[trn_idx, -4]
        y_trainy = data.iloc[trn_idx, -3]
        y_trainf = data.iloc[trn_idx, -2]

        X_valid = data.iloc[val_idx, :-4]
        y_validx = data.iloc[val_idx, -4]
        y_validy = data.iloc[val_idx, -3]
        y_validf = data.iloc[val_idx, -2]

        # modelx = lgb.LGBMRegressor(**lgb_params)
        # modelx = xgb.XGBRegressor(**xgb_params)
        modelx = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
        # modelx = SVR(kernel='linear', C=100, gamma='auto')
        # modelx = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1, coef0=1)
        with timer("fit X"):
            modelx.fit(X_train, y_trainx,
                       # eval_set=[(X_valid, y_validx)],
                       # eval_metric='rmse',
                       #verbose=False,
                       #early_stopping_rounds=20
                      )

        # modely = lgb.LGBMRegressor(**lgb_params)
        # modely = xgb.XGBRegressor(**xgb_params)
        modely = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
        # modely = SVR(kernel='linear', C=100, gamma='auto')
        # modely = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1, coef0=1)
        with timer("fit Y"):
            modely.fit(X_train, y_trainy #,
                       # eval_set=[(X_valid, y_validy)],
                       # eval_metric='rmse',
                       # verbose=False,
                       # early_stopping_rounds=20
                       )

        # modelf = lgb.LGBMClassifier(**lgb_f_params)
        # remove param eval_metric='multi_logloss', for xgb
        # modelf = xgb.XGBClassifier(**xgb_f_params)
        modelf = SVC(decision_function_shape='ovo')
        with timer("fit F"):
            modelf.fit(X_train, y_trainf #,
                       # eval_set=[(X_valid, y_validf)],
                       # eval_metric='mlogloss', #for xgboot
                       # verbose=False,
                       # early_stopping_rounds=20
                       )

        oof_x[val_idx] = modelx.predict(X_valid)
        oof_y[val_idx] = modely.predict(X_valid)
        oof_f[val_idx] = modelf.predict(X_valid).astype(int)
        
        with timer("Pred X"):
            preds_x += modelx.predict(test_data.iloc[:, :-1]) / N_SPLITS
            
        with timer("Pred Y"):
            preds_y += modely.predict(test_data.iloc[:, :-1]) / N_SPLITS
        
        with timer("Pred F"):
            preds_f_arr[:, fold] = modelf.predict(test_data.iloc[:, :-1]).astype(int)

        score = comp_metric(oof_x[val_idx], oof_y[val_idx], oof_f[val_idx],
                            y_validx.to_numpy(), y_validy.to_numpy(), y_validf.to_numpy())
        print(f"fold {fold}: mean position error {score}")
        score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, fold, SEED, score)
        
        break # for test purpose otherwise remove this break statement

    print("*+"*40)
    print(f"file #{n_files}, shape={data.shape}, name={os.path.basename(file)}")
    score = comp_metric(oof_x, oof_y, oof_f,
                        data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy(), data.iloc[:, -2].to_numpy())
    oof.append(score)
    print(f"mean position error {score}")
    print("*+"*40)
    score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, 999, SEED, score)

    preds_f_mode = stats.mode(preds_f_arr, axis=1)
    preds_f = preds_f_mode[0].astype(int).reshape(-1)
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    test_preds.columns = subm.columns
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    predictions.append(test_preds)
    break # for test purpose otherwise remove this break statement

# ------------------------------------------------------------------------------
# Submit the result
# ------------------------------------------------------------------------------
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)
all_preds.to_csv('submission.csv')

There are 24 *_train.csv files
There are 24 *_test.csv files
process training set for site file num 0
process testing set for site file num 0


[0.8GB(-0.0GB): 1875.343sec] fit X
[0.8GB(+0.0GB): 2011.414sec] fit Y
[1.0GB(+0.1GB): 64.917sec] fit F
[1.2GB(+0.0GB): 17.939sec] Pred X
[1.2GB(+0.0GB): 17.843sec] Pred Y
[1.2GB(+0.0GB): 3.309sec] Pred F


fold 0: mean position error 63.214866311881686
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
file #0, shape=(9296, 3401), name=5a0546857ecc773753327266_train.csv
mean position error 155.8233074427557
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
